### Imports

In [ ]:
import utils
import re

#### Import the saved variables from the previous notebook.

In [ ]:
%store -r

### Comparing results with Amazon OpenSearch Service

To understand how results are ranked, you can run queries with and without personalization, and compare the results. You can use the following Python code to run two different queries and output the results to two JSON files. The first method runs a query to generate results without personalization. The second runs a method that uses the plugin to re-rank results

In [ ]:
selected_weight=0.5
user_id = "12" # Here we select a user id to compare the results
search = "Tom Cruise" # We use a search Term to search our movies index which we created earlier

query1 = """{
            "size": 20,
            "query": {
                "multi_match": {
                    "query": "%SearchText%",
                    "fields": ["title", "plot", "genres", "directedBy", "starring"]
                }
            }
        }"""

query2 = """{
            "size": 20,
            "query": {
                "multi_match": {
                    "query": "%SearchText%",
                    "fields": ["title", "plot", "genres", "directedBy", "starring"]
                }
            },
            "ext": {
                "personalize_request_parameters": {
                    "user_id": "%UserId%"
                }
            }
        }"""



We will fill in the placeholders in the query with the actual variable values that were defined earlier.

In [ ]:
doc_str = re.sub(r"%SearchText%", search, query1)
doc_str = re.sub(r"%UserId%", user_id, doc_str)

doc_str_2 = re.sub(r"%SearchText%", search, query2)
doc_str_2 = re.sub(r"%UserId%", user_id, doc_str_2)

We will load the interactions and items datasets into dataframes. These will be used to identify recent movies the user has watched, based on their interaction history.

In [ ]:
interactions_df = utils.get_interactions(root_dir)
items_df = utils.get_items(root_dir)

### Recent interactions for a user

In [ ]:
utils.load_recent_movies(user_id, interactions_df, items_df)

### Execute the search queries against the OpenSearch domain.
We will execute two queries - one without personalization and one with personalization - and compare the results. The first query is run without any personalization parameters. The second query is run by setting the personalize_request_parameters to the user_id as shown in the query 2 above, below is an excerpt.

`"ext": {
        "personalize_request_parameters": {
            "user_id": "%UserId%"
        }
    }`


This allows us to evaluate the difference when applying personalization versus not applying personalization. We created the search pipeline in the [2.Configure_Amazon_OpenSearch.ipynb](./2.Configure_Amazon_OpenSearch.ipynb)

In [ ]:
res1 = utils.run_search(doc_str, HOST)

Now we run the same query but this time we pass additional parameters such as the userid and the pipeline name to get personalized search results. Feel free to check the `run_search` in `utils.py` method which runs the actual query

In [ ]:
res2 = utils.run_search(doc_str_2, HOST, "intelligent_ranking")

In [ ]:
utils.compare_results(res1, res2)

#### Performing the search again with a modified weight parameter for the search pipeline
You can experiment with different values of weight in the below cells and see the effect of Personalization on your queries.

Updating the weight for the search pipeline (can be set from 0.0 to 1.0).

In [ ]:
utils.update_pipeline("intelligent_ranking", "1.0", campaign_arn, role_arn_for_personalize, region, HOST, PORT)


Re-executing the search query with an updated weight parameter configured for the search pipeline.

In [ ]:
res2 = utils.run_search(doc_str_2, HOST, "intelligent_ranking")

Comparing results

In [ ]:
utils.compare_results(res1, res2)

The weight parameter controls the balance between the OpenSearch relevance ranking and the Amazon Personalize personalized ranking. When the weight is set to 0.0, no personalization from Amazon Personalize is applied - the results are ranked solely based on OpenSearch relevance. 

As the weight is increased towards 1.0, more priority is given to the Amazon Personalize ranking scores over the OpenSearch relevance scores. A weight of 1.0 means the final ranking will be fully determined by the Amazon Personalize personalized ranking.

So the closer the weight parameter is to 1.0, the more the results ranking will be biased towards the personalized ranking from Amazon Personalize over the relevance ranking from OpenSearch. This allows you to tune the level of personalization to find the right balance for your application.

Comparing results with weight = 0.0 and weight = 1.0 clearly shows the two extremes of how the weighting parameter controls the influence of Amazon Personalize on the final ranked results returned to the user.